## Import 
*Note: The environment has semantic-link should be installed already*

In [5]:
import sempy.fabric as fabric
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.functions import col
from pyspark.sql.types import *
import uuid
import datetime

StatementMeta(, 3c7032fb-79cb-42fe-ac6f-9aaf0b086655, 9, Finished, Available)

## Set Branch Parameter

In [3]:
branch = "main"


StatementMeta(, 3c7032fb-79cb-42fe-ac6f-9aaf0b086655, 7, Finished, Available)

## Retrieve DAX Queries

In [4]:
# Generate Run Guid
run_uuid = str(uuid.uuid4())
# Generate Timestamp
run_dt = datetime.datetime.now()

# Get latest queries for the branch
# NOTE: Switch to commit id for orchestration
dax_df = spark.sql("SELECT * FROM Continuous_Integration.dax_queries WHERE Azure_DevOps_Branch_Name = '" + branch + "' AND Dataset_Sub_Folder_Path LIKE '%.Tests.dax' AND Timestamp = (SELECT MAX(Timestamp) FROM Continuous_Integration.dax_queries)")

display(dax_df)


StatementMeta(, 3c7032fb-79cb-42fe-ac6f-9aaf0b086655, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, 6460141b-683c-42fa-8272-ac9a607e608b)

In [5]:
# Create empty schema
test_schema = StructType(
                    [StructField('Run_GUID',
                                StringType(), True),
                    StructField('Test_Name',
                                StringType(), True),
                    StructField('Expected_Value',
                                  StringType(), True),
                    StructField('Actual_Value',
                                StringType(), True),
                    StructField('Passed',
                                BooleanType(), True),
                    StructField('Concatenated_Key',
                                StringType(), True),
                    StructField('Timestamp',
                                TimestampType(), True)])                                



StatementMeta(, 2b2abfa6-d9b0-490a-8afc-6a7054991eeb, 12, Finished, Available)

### Run Tests
Iterates through DAX Queries, runs the tests and saves the results to test_results.

In [6]:

# Itertuples
dax_df2 = dax_df.toPandas()
# Possible a faster way, but this works for now
for row in dax_df2.itertuples():
    dqs = fabric.evaluate_dax(row.Dataset_Name,row.DAX_Queries, row.Workspace_GUID)
    # Retrieve
    test = dqs[['[TestName]','[ExpectedValue]','[ActualValue]','[Passed]']]
    # Set Concatenated Key
    test['Concatenated_Key'] = row.Concatenated_Key
    test['Run_GUID'] = run_uuid
    test['Test_Name'] = test['[TestName]']
    test['Expected_Value'] = test['[ExpectedValue]']
    test['Actual_Value'] = test['[ActualValue]']
    test['Passed'] = test['[Passed]']
    test['Timestamp'] = run_dt

    test_results = test[['Run_GUID','Test_Name','Expected_Value','Actual_Value','Passed','Concatenated_Key','Timestamp']]
    display(test_results)
    test_results.to_lakehouse_table("test_results", "append", test_schema)



StatementMeta(, 2b2abfa6-d9b0-490a-8afc-6a7054991eeb, 13, Finished, Available)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:428: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Expected a string or bytes dtype, got bool
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


SynapseWidget(Synapse.DataFrame, 12a612fa-036e-47b8-a7dc-e4feff6b0315)

/nfs4/pyenv-59429dc0-8414-4d19-b8f4-93f87b824846/lib/python3.10/site-packages/sempy/_metadata/_mdataframe.py:193: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result = attr(*args, **kwargs)
/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:428: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Expected a string or bytes dtype, got bool
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


SynapseWidget(Synapse.DataFrame, c861c29a-79ec-41d6-a60d-161108c00c0d)

/nfs4/pyenv-59429dc0-8414-4d19-b8f4-93f87b824846/lib/python3.10/site-packages/sempy/_metadata/_mdataframe.py:193: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result = attr(*args, **kwargs)


## Output Test Results

In [7]:
df = spark.sql("SELECT * FROM test_results LIMIT 1000")
display(df)

StatementMeta(, 2b2abfa6-d9b0-490a-8afc-6a7054991eeb, 14, Finished, Available)

SynapseWidget(Synapse.DataFrame, 59a82b92-1c79-4a91-b9b1-45482f19a5fb)